# WHEN OBJECTS ARE ALIKE

In the programming world, duplicate code is considered evil.
 
We should not have multiple copies of the same, or similar, code in different places. 

When we fix a bug in one copy and fail to fix the same bug in another copy, we've caused no end of problems for ourselves.



## INHERITANCE

Technically every class we create uses inheritance.

All Python classes are subclasses of the special class named `object`.

If we don't explicitly inherit from a different class, our classes will automatically inherit from object.

All classes automatically inherit from object if we don't explicitly provide a different superclass (parent class).

 Inheritance requires a minimal amount of extra syntax over a basic class definition. 
 
Simply include the name of the parent class inside parentheses between the class name and the colon that follows.
 
This is all we have to do to tell Python that the new class should be derived from the given superclass.

### How To Apply Inheritance

The simplest and most obvious use of inheritance is to add functionality to an existing class.

Let's start with a contact manager that tracks names and email addresses.

The `contact` class is responsible for maintaining global list of all contacts ever seen in a class variable, and for initializing the name and address for an individual contact:

In [61]:
class Contact:
    
    all_contacts: list["Contact"] = []
    
    def __init__(self, name: str = "" , email: str= "" ) -> None:
        self.name = name
        self.email = email
        Contact.all_contacts.append(self)
        
    def __repr__(self) -> str:
        return (
            f"{self.__class__.__name__}("
            f"{self.name!r}, {self.email!r}"
            f")"
      )
        
        
    

This example introduces us to class variables.

The `all_contacts` list,because it is part of the class definition, is shared by all instances of this class.

This means that there is only one `Contact.all_contacts` list.

We can also access it as `self.all_contacts` from within any method on an instance of the `Contact` class.

If a field cannot be found on the object via `self`, then it will be found on the class and will thus refer to the same single list.




#### PRO-TIP

Be careful with the `self`-based reference.

It can only provide access to an existing class-based variable.

If you evert attempt to set the variable using `self.all_contacts`, you will actually be creating a new instance variable associated just with that object.

The class variable will still be unchanged and accesible as `Contact.all_contacts`

In [62]:
c_1 = Contact("Sevval", "sevvalunver@gmail.com")
c_2 = Contact("Serdar", "serdarunver@gmail.com")

In [63]:
Contact.all_contacts

[Contact('Sevval', 'sevvalunver@gmail.com'),
 Contact('Serdar', 'serdarunver@gmail.com')]

This is a simple class that allows us to track a couple of pieces of data about each contact. 

But what if some of our contacts are also suppliers that we need to order supplies from? 

We could add an `order` method to the Contact class, but that would allow people to accidentally order things from contacts who are customers or family friends. 

Instead, let's create a new `Supplier` class that acts like our `Contact` class, but has an additional `order` method that accepts a yet-to-be-defined `Order` object:

In [64]:
class Supplier(Contact):
    
    def order(self, order: "Order") -> None:
        print(
            "If this were a real system we would send "
            f"'{order}' order to '{self.name}'"
        )

If we test this class in our trusty interpreter, we see that all contacts, including suppliers, accept a name and email address in their ``__init__`` method, but that only `suppliers` instance have an `order` method:

In [65]:
c = Contact("Some Body", "somebody@somename.com")
s = Supplier("Sup Plier", "supplier@example.com")

In [66]:
print(c.name, c.email, s.name, s.email)

Some Body somebody@somename.com Sup Plier supplier@example.com


In [67]:
from pprint import pprint
pprint(c.all_contacts)

[Contact('Sevval', 'sevvalunver@gmail.com'),
 Contact('Serdar', 'serdarunver@gmail.com'),
 Contact('Some Body', 'somebody@somename.com'),
 Supplier('Sup Plier', 'supplier@example.com')]


In [68]:
c.order("I need pliers")

AttributeError: 'Contact' object has no attribute 'order'

In [72]:
s.order("I need pliers")

If this were a real system we would send 'I need pliers' order to 'Sup Plier'


Supplier class can do everything a Contact can do (including adding itself to the list of Contact.all_contacts), but it also has an additional method that allows us to place orders.

If we used `self.all_contacts` then this would not collect all objects into the `Contact` class, but would put `Supplier` instances into `Supplier.all_contacts`. 

## Extending Built-Ins

One interesting use of inheritance is adding functionality to built-in types (classes).

In the `Contact` class seen earlier, we are adding contacts to a list of all contacts.

What if we wanted to search that list by name?

We could add a metgod on the `Contact` class to do that, but it feels like this method actually belongs to the list itself.

We need to import `annotations` module from `__future__` package.


In [73]:
from __future__ import annotations

class ContactList(list["Contact"]):
    
    def search(self, name: str) -> list["Contact"]:
        """Return all contacts that contain the search value in their name."""
        matching_contacts: list["Contact"] = []
        for contact in self:
            if name in contact.name:
                matching_contacts.append(contact)
        return matching_contacts
    
class Contact:
    
        all_contacts: ContactList = ContactList()
        
        def __init__(self, name: str = "", email: str = "") -> None:
            self.name = name
            self.email = email
            Contact.all_contacts.append(self)
            
        def __repr__(self) -> str:
            return (
                f"{self.__class__.__name__}("
                f"{self.name!r}, {self.email!r}"
                f")"
            )

In [74]:
c1 = Contact("John A", "johna@example.com")
c2 = Contact("John B", "johnb@example.com")
c3 = Contact("Jenna C", "cutty@example.com")

In [75]:
[c.name for c in Contact.all_contacts.search("John")]

['John A', 'John B']

Instead of instatiating a gneric list as our class variable, we create a new `ContactList` class that extends the built-in `list` class.

As a second example, we can extend the `dict` class, which is a collection of keys and their associated values.

We can create instances of dictionaries using `{}` syntax.

An extended class that tracts the longest ket it has seen:

In [76]:
class LongNameDict(dict[str, int]):
    
    def longest_key(self) -> Optional[str]:
        longest = None
        for key in self:
            if longest is None or len(key) > len(longest):
                longest = key
        return longest

The hint for the class narrowed the generic dict to a more specific dict[str, int]; the keys are of type str and the values are of type int.
 
This helps mypy reason about the longest_key() method. Since the keys are supposed to be str-type objects, the statement for key in self: will iterate over str objects. 

The result will be a str, or possibly None. That's why the result is described as `Optional[str]`.

We're going to be working with strings and integer values. Perhaps the strings are usernames, and the integer values are the number of articles they've read on a website.
 
In addition to the core username and reading history, we also need to know the longest name so we can format a table of scores with the right size display box.

In [77]:
articles_read = LongNameDict()

articles_read["Doug"] = 34
articles_read["Anne"] = 11
articles_read["Boris"] = 42

In [78]:
articles_read.longest_key()

'Boris'

Most built-in types can be similarly extended. These built-in types fall into several interesting families, with separate kinds of type hints:

### 1-) Generic Collections:

`set`,`list`,`dict`.

These use type hints like `set[something]`, `list[something]`, and `dict[key,value]` to narrow the hint from purely generic to something more specific that the application will actually use.

To use the generic types as annotations, a `from __future__ import annotations` is required.

### 2-)

The `typing.NamedTuple` definition lets us define new kinds of immutable tuples and provide useful names for the members.

### 3-)

Python has type hints for file-related I/O objects. A new kind of file can use a type hint of `typing.TextIO` or `typing.BinaryIO` to describe built-file file operations.

### 4-)

It's possible to create new types of strings by extending `typing.Text`. For the most part, the built-in `str` class does everything we need.

### 5-)

New numeric types often start with the `numbers` module as a source for built-in numeric functionality.

## Overriding and Super

Our `Contact` class allows only a name and an email address. This may be sufficient for most contacts, but what if we want to add a phone number for our close friends?

We could do this easily by setting a `phone` attribute on the contact after it is constructed.

But if we want to make this third variable available on initialization, we have to override the `__init__` method from the superclass.

Overriding means altering or replacing a method of the superclass with a new method in the subclass.

No special syntax is needed to do this; the subclass's newly created method is automatically called instead of the superclass's method.

In [79]:
class Friend(Contact):
    
    def __init__(self, name: str = "", email: str = "", phone: str = "") -> None:
        self.name = name
        self.email = email
        self.phone = phone

Our Contact and Friend classes have duplicate code to set up the name and email properties; this can make code maintenance complicated, as we have to update the code in two or more places. 

More alarmingly, our Friend class is neglecting to add itself to the all_contacts list we have created on the Contact class. 

Finally, looking forward, if we add a feature to the Contact class, we'd like it to also be part of the Friend class.

What we really need is a way to execute the original __init__() method on the Contact class from inside our new class. 

This is what the super() function does; it returns the object as if it was actually an instance of the parent class, allowing us to call the parent method directly:


In [80]:
class Friend(Contact):
    
    def __init__(self, name: str = "", email: str = "", phone: str = "") -> None:
        super().__init__(name, email)
        self.phone = phone

This example first binds the instance to the parent class using `super()` and calls `__init__()` on that object, passing in the expected arguments.

It then does its own initialization, namely, setting the `phone` attribute, which is unique to the `Friend` class.

The `Contact` class provided a definition for the `__repr__()` method to produce a string representation of the object.

Our class did not override the `__repr__()` method inherited from the superclass.

In [81]:
f = Friend("Sevvalism26", "ss@example", "12345")

In [84]:
Contact.all_contacts

[Contact('John A', 'johna@example.com'),
 Contact('John B', 'johnb@example.com'),
 Contact('Jenna C', 'cutty@example.com'),
 Friend('Sevvalism26', 'ss@example')]

The details shown for a Friend instance don't include the new attribute. 

It's easy to overlook the special method definitions when thinking about class design.

A super() call can be made inside any method. Therefore, all methods can be modified via overriding and calls to super(). 

The call to super() can also be made at any point in the method; we don't have to make the call as the first line. 

For example, we may need to manipulate or validate incoming parameters before forwarding them to the superclass. 

# Multiple Inheritance

Multiple inheritance in principle is simple: a subclass that inherits from more than one parent class can access functionality from both of them.

In practice, it requires some care to be sure any method overrides are fully understood.

The simplest and most useful form of multiple inheritance follows a design pattern called the **mixin**.

A mixin class definition is not intended to exist on its own; but is meant to be inherited by some other class to provide extra functionality.

For example, let's say we wanted to add functionality to our `Contact` class that allows sending an email to `self.email`.

Sending mail is a common task that we might want to use on many other classes. So, we can write a simple mixin class that provides this functionality:


In [85]:
from typing import Protocol


class Emailable(Protocol):
    email: str
    
class MailSender(Emailable):
    
    def send_mail(self, message: str) -> None:
        print(f"Sending mail to {self.email}")
        # Add e-mail logic here

The `MailSender` class does not do anything special.

We have two classes because we are describing two things: aspects of the host class for a mixin, and new aspects the mixin provides to the host.

We needed to create a hint, `Emailable`, to describe the kinds of classes or `MailSender` mixin expects to work with.

This kind of type hint is called a `Protocol`.

`Protocols` generally have methods and can also have a class-level attribute names with type hints but not full assignment statements.

A protocol tells that any class (or Subclass) of `Emailable` object must support an `email` attribute of type `str`.

Note that we are relying on Python's name resolution rules.

The name `self.email` can be resolved as either an instance variable, or a class-level variable, `Emailable.email`, or a property.

The mypy tool will check all the classes mixed in with `MailSender` for instance- or class-level definitions. 

We only need to provide the name of the attribute at the class level, with a type hint to make it clear to mypy that the mixin does not define the attribute – the class into which it's mixed will provide the `email` attribute.

Because of Python's duck typing rules, we can use the `MailSender` mixin with any class that has an `email` attribute defined. 

A class with which `MailSender` is mixed doesn't have to be a formal subclass of `Emailable`; it only has to provide the required attribute.

For brevity, we didn't include the actual email logic here; if you're interested in studying how it's done, see the `smtplib` module in the Python standard library.

The `MailSender` class does allow us to define a new class that describes both a `Contact` and a `MailSender`, using multiple inheritance:

In [86]:
class EmailableContact(Contact, MailSender):
    pass


The syntax for multiple inheritance looks like a parameter list in the class definition. 

Instead of including one base class inside the parentheses, we include two (or more), separated by a comma. 

When it's done well, it's common for the resulting class to have no unique features of its own. 

It's a combination of mixins, and the body of the class definition is often nothing more than the pass placeholder.

We can test this new hybrid to see the mixin at work:

In [87]:
e = EmailableContact("John Smith", "jognb@sloop.net")

In [88]:
Contact.all_contacts

[Contact('John A', 'johna@example.com'),
 Contact('John B', 'johnb@example.com'),
 Contact('Jenna C', 'cutty@example.com'),
 Friend('Sevvalism26', 'ss@example'),
 EmailableContact('John Smith', 'jognb@sloop.net')]

In [89]:
e.send_mail("Hello, test e-mail here")

Sending mail to jognb@sloop.net


The `Contact` initializer is still adding the new contact to the `all_contacts` list, and the mixin is able to send mail to `self.email`, so we know that everything is working.

let's consider some other options we had for this example, rather than using a mixin:

We could have used single inheritance and added the `send_mail` function to a subclass of `Contact`. 

The disadvantage here is that the email functionality then has to be duplicated for any unrelated classes that need an email. 

For example, if we had email information in the payments part of our application, unrelated to these contacts, and we wanted a `send_mail()` method, we'd have to duplicate the code.

We can create a standalone Python function for sending an email, and just call that function with the correct email address supplied as a parameter when the email needs to be sent (this is a very common choice). 

Because the function is not part of a class, it's harder to be sure that proper encapsulation is being used.

We could explore a few ways of using composition instead of inheritance. 

For example, `EmailableContact` could have a `MailSender` object as a property instead of inheriting from it. 

This leads to a more complex `MailSender` class because it now has to stand alone. It also leads to a more complex `EmailableContact` class because it has to associate a `MailSender` instance with each `Contact`.


We could try to monkey patch (we'll briefly cover monkey patching in Chapter 13, Testing Object-Oriented Programs) the `Contact` class to have a `send_mail` method after the class has been created. 

This is done by defining a function that accepts the `self` argument, and setting it as an attribute on an existing class. 

This is fine for creating a unit test fixture, but terrible for the application itself.
